In [67]:
import pandas as pd
# We'll need numpy, as usual
import numpy as np

# Import natural language toolkit
import nltk

# Importing some data
train=pd.read_csv('/Users/hechijian/Desktop/600/trainingandtestdata/training.1600000.processed.noemoticon.csv',encoding='iso-8859-1',header=None)
# test=pd.read_csv('/Users/hechijian/Desktop/trainingandtestdata/testdata.manual.2009.06.14.csv',encoding='iso-8859-1',header=None)
# f2=pd.read_csv('/Users/hechijian/Desktop/Comments.csv')
# Getting review content with pos/neg for each
documents=[]
for i in range(len(train)):
    documents.append((train.get_value(i,5)))
documents = [(list(movie_reviews.words(fileid)), category)
                for category in movie_reviews.categories()
                    for fileid in movie_reviews.fileids(category)]

/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [2]:
train.iloc[:,0].value_counts()
# train.get_value(800000,0)

4    800000
0    800000
Name: 0, dtype: int64

In [3]:
from nltk.corpus import stopwords
import string
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
documents_raw=[]
comments_raw=[]
catergories_raw=[]
for i in range(100000):
    if train.get_value(i,0)==0:
        documents_raw.append((tknzr.tokenize(train.get_value(i,5)),'pos'))
    else:
        documents_raw.append((tknzr.tokenize(train.get_value(i,5)),'neg'))
for i in range(900000,1000000):
    if train.get_value(i,0)==0:
        documents_raw.append((tknzr.tokenize(train.get_value(i,5)),'pos'))
    else:
        documents_raw.append((tknzr.tokenize(train.get_value(i,5)),'neg'))


/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [4]:
stop=stopwords.words('english')
documents=[]
for i in range(len(documents_raw)):
    tem=[]
    tem=[w for w in documents_raw[i][0] 
                   if w.lower() not in stop 
                   and w not in string.punctuation]
    documents.append((tem,documents_raw[i][1]))
    

In [5]:
all_words=[]
for i in range(len(documents)):
    for j in documents[i][0]:
        all_words.append(j)
    
all_words_final = nltk.FreqDist(w.lower() for w in all_words)


In [6]:
word_features = [w for (w,c) in all_words_final.most_common(500)]

In [7]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features
train_set = [(document_features(d),c) for (d,c) in documents]
clf = nltk.NaiveBayesClassifier.train(train_set)



In [9]:
f2=pd.read_csv('/Users/hechijian/Desktop/600/Tweets.csv')

documents_Tweets=[]
documents_Tweets_final=[]
for i in range(len(f2)):
    documents_Tweets.append(tknzr.tokenize(f2.get_value(i,'text')))
for i in range(len(documents_Tweets)):
    tem=[]
    tem=[w for w in documents_Tweets[i] 
                   if w.lower() not in stop 
                   and w not in string.punctuation]
    documents_Tweets_final.append(tem)
test_set=[document_features(d) for d in documents_Tweets_final]



/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [22]:
f3=pd.read_csv('/Users/hechijian/Desktop/600/Comments.csv')
#f3=f3.dropna()

documents_Comments=[]
documents_Comments_final=[]

for i in range(len(f3)):
    documents_Comments.append(tknzr.tokenize(f3.get_value(i,'text')))
for i in range(len(documents_Comments)):
    tem=[]
    tem=[w for w in documents_Comments[i] 
                   if w.lower() not in stop 
                   and w not in string.punctuation]
    documents_Comments_final.append(tem)
test_set=[document_features(d) for d in documents_Comments_final]


/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [25]:
result_Tweets=[]
for i in range(len(f2)):
    result_Tweets.append((f2.get_value(i,'id'),clf.classify(test_set[i])))

    
result_Comments=[]
for i in range(len(f3)):
    result_Comments.append((f3.get_value(i,'reply_id'),clf.classify(test_set[i])))

        


/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hechijian/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [48]:
reply_id=[]
result_Comments_final=[]
for i in range(len(result_Comments)):
    if result_Comments[i][0]>100:
        reply_id.append(result_Comments[i][0])
        result_Comments_final.append(result_Comments[i])
a=list(set(reply_id))

181

In [64]:
Comments_neg={}
Comments_pos={}
for i in result_Comments_final:
    Comments_pos[i[0]]=0
    Comments_neg[i[0]]=0
for i in result_Comments_final:
    if i[1]=='pos':
        Comments_pos[i[0]]+=1
    else:
        Comments_neg[i[0]]+=1
        

In [65]:
Comments_neg

{1.0642169566797169e+18: 1173,
 1.0642092461144596e+18: 215,
 1.0642457107475908e+18: 463,
 1.0642467871611452e+18: 180,
 1.0642274831873188e+18: 198,
 1.0639871576504852e+18: 89,
 1.0630469738577184e+18: 7,
 1.05506448464409e+18: 0,
 5.530812688006144e+17: 7,
 1.0638339807205048e+18: 14,
 1.0635934565793546e+18: 25,
 1.0638347334302065e+18: 54,
 1.0616506541947452e+18: 1,
 1.0637581072292576e+18: 13,
 1.0630387336735252e+18: 13,
 1.063591994738516e+18: 12,
 1.0638347295840544e+18: 35,
 7.880314479322153e+17: 0,
 1.0630425858020392e+18: 13,
 1.0635934596956692e+18: 26,
 1.0623117857877444e+18: 4,
 1.0618405110262988e+18: 0,
 1.0621530514594692e+18: 2,
 1.0619570742534184e+18: 1,
 1.0630840732315444e+18: 6,
 1.0631683419410636e+18: 1,
 1.063591735169872e+18: 2,
 1.0619543258434684e+18: 0,
 1.0635175482584474e+18: 14,
 1.0638359366040904e+18: 18,
 1.0619526048072744e+18: 0,
 1.0619628693765408e+18: 0,
 1.061168803218948e+18: 3,
 1.062371844836262e+18: 8,
 1.063081553075626e+18: 17,
 5.16

In [66]:
Comments_pos

{1.0642169566797169e+18: 502,
 1.0642092461144596e+18: 81,
 1.0642457107475908e+18: 293,
 1.0642467871611452e+18: 70,
 1.0642274831873188e+18: 129,
 1.0639871576504852e+18: 43,
 1.0630469738577184e+18: 5,
 1.05506448464409e+18: 1,
 5.530812688006144e+17: 7,
 1.0638339807205048e+18: 8,
 1.0635934565793546e+18: 13,
 1.0638347334302065e+18: 22,
 1.0616506541947452e+18: 0,
 1.0637581072292576e+18: 12,
 1.0630387336735252e+18: 1,
 1.063591994738516e+18: 5,
 1.0638347295840544e+18: 10,
 7.880314479322153e+17: 1,
 1.0630425858020392e+18: 8,
 1.0635934596956692e+18: 17,
 1.0623117857877444e+18: 2,
 1.0618405110262988e+18: 2,
 1.0621530514594692e+18: 2,
 1.0619570742534184e+18: 0,
 1.0630840732315444e+18: 8,
 1.0631683419410636e+18: 0,
 1.063591735169872e+18: 3,
 1.0619543258434684e+18: 1,
 1.0635175482584474e+18: 7,
 1.0638359366040904e+18: 7,
 1.0619526048072744e+18: 2,
 1.0619628693765408e+18: 1,
 1.061168803218948e+18: 3,
 1.062371844836262e+18: 4,
 1.063081553075626e+18: 9,
 5.163821777986